# ETL Pipeline

### Imports

In [1]:
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

### Constants

In [2]:
MESSAGES_FILEPATH = os.path.join("..", "data", "messages.csv")
CATEGORIES_FILEPATH = os.path.join("..", "data", "categories.csv")
DATA_OUTPUT_FILENAME = "Disasters.db"
SQL_TABLE_NAME = "DisastersData"

### Loading Data

In [3]:
df_messages = pd.read_csv(MESSAGES_FILEPATH)
df_messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
df_categories = pd.read_csv(CATEGORIES_FILEPATH)
df_categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### Data Wrangling

#### Merging messages and categories

In [5]:
df_data = pd.merge(df_messages, df_categories, on="id")
df_data.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


####  Splitting  `categories` into separate category columns.

In [6]:
df_categories_new = df_data["categories"].str.split(pat=';', expand=True)
df_categories_new.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [7]:
np.unique(df_categories_new.values)

array(['aid_centers-0', 'aid_centers-1', 'aid_related-0', 'aid_related-1',
       'buildings-0', 'buildings-1', 'child_alone-0', 'clothing-0',
       'clothing-1', 'cold-0', 'cold-1', 'death-0', 'death-1',
       'direct_report-0', 'direct_report-1', 'earthquake-0',
       'earthquake-1', 'electricity-0', 'electricity-1', 'fire-0',
       'fire-1', 'floods-0', 'floods-1', 'food-0', 'food-1',
       'hospitals-0', 'hospitals-1', 'infrastructure_related-0',
       'infrastructure_related-1', 'medical_help-0', 'medical_help-1',
       'medical_products-0', 'medical_products-1', 'military-0',
       'military-1', 'missing_people-0', 'missing_people-1', 'money-0',
       'money-1', 'offer-0', 'offer-1', 'other_aid-0', 'other_aid-1',
       'other_infrastructure-0', 'other_infrastructure-1',
       'other_weather-0', 'other_weather-1', 'refugees-0', 'refugees-1',
       'related-0', 'related-1', 'related-2', 'request-0', 'request-1',
       'search_and_rescue-0', 'search_and_rescue-1', 'secu

#### Extracting category names

In [8]:
categories = df_categories_new.iloc[0,:].apply(lambda x:x[:-2])
categories

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

#### Filling column names

In [9]:
df_categories_new.columns = ["category_{}".format(c) for c in categories]
df_categories_new.head()

,category_related,category_request,category_offer,category_aid_related,category_medical_help,category_medical_products,category_search_and_rescue,category_security,category_military,category_child_alone,...,category_aid_centers,category_other_infrastructure,category_weather_related,category_floods,category_storm,category_fire,category_earthquake,category_cold,category_other_weather,category_direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


#### Changing values to integers

In [10]:
for col in df_categories_new.columns:
    df_categories_new[col] = df_categories_new[col].str[-1].astype(np.int)

df_categories_new.head()

,category_related,category_request,category_offer,category_aid_related,category_medical_help,category_medical_products,category_search_and_rescue,category_security,category_military,category_child_alone,...,category_aid_centers,category_other_infrastructure,category_weather_related,category_floods,category_storm,category_fire,category_earthquake,category_cold,category_other_weather,category_direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Replacing `categories` column in `df_data` with new category columns

In [11]:
df_data = df_data.drop("categories", axis=1)
df_data.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [12]:
df_data = pd.concat([df_data, df_categories_new], axis=1)
df_data.head()

,id,message,original,genre,category_related,category_request,category_offer,category_aid_related,category_medical_help,category_medical_products,...,category_aid_centers,category_other_infrastructure,category_weather_related,category_floods,category_storm,category_fire,category_earthquake,category_cold,category_other_weather,category_direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Remove duplicates

In [13]:
df_data.duplicated().sum()

170

In [14]:
df_data.drop_duplicates(inplace=True)

In [15]:
df_data.duplicated().sum()

0

#### Removing columns with too many null values

In [16]:
df_data.isnull().mean()

id                                 0.000000
message                            0.000000
original                           0.612069
genre                              0.000000
category_related                   0.000000
category_request                   0.000000
category_offer                     0.000000
category_aid_related               0.000000
category_medical_help              0.000000
category_medical_products          0.000000
category_search_and_rescue         0.000000
category_security                  0.000000
category_military                  0.000000
category_child_alone               0.000000
category_water                     0.000000
category_food                      0.000000
category_shelter                   0.000000
category_clothing                  0.000000
category_money                     0.000000
category_missing_people            0.000000
category_refugees                  0.000000
category_death                     0.000000
category_other_aid              

In [17]:
drop_threshold = 0.6
df_data = df_data.loc[:, df_data.isnull().mean() < drop_threshold]
df_data = df_data.reset_index(drop=True)

In [18]:
df_data.isnull().mean()

id                                 0.0
message                            0.0
genre                              0.0
category_related                   0.0
category_request                   0.0
category_offer                     0.0
category_aid_related               0.0
category_medical_help              0.0
category_medical_products          0.0
category_search_and_rescue         0.0
category_security                  0.0
category_military                  0.0
category_child_alone               0.0
category_water                     0.0
category_food                      0.0
category_shelter                   0.0
category_clothing                  0.0
category_money                     0.0
category_missing_people            0.0
category_refugees                  0.0
category_death                     0.0
category_other_aid                 0.0
category_infrastructure_related    0.0
category_transport                 0.0
category_buildings                 0.0
category_electricity     

#### Checking for columns with the same value

In [19]:
for col in df_categories_new.columns:
    print("Col '{}' unique values: {}".format(col, df_data[col].nunique()))

Col 'category_related' unique values: 3
Col 'category_request' unique values: 2
Col 'category_offer' unique values: 2
Col 'category_aid_related' unique values: 2
Col 'category_medical_help' unique values: 2
Col 'category_medical_products' unique values: 2
Col 'category_search_and_rescue' unique values: 2
Col 'category_security' unique values: 2
Col 'category_military' unique values: 2
Col 'category_child_alone' unique values: 1
Col 'category_water' unique values: 2
Col 'category_food' unique values: 2
Col 'category_shelter' unique values: 2
Col 'category_clothing' unique values: 2
Col 'category_money' unique values: 2
Col 'category_missing_people' unique values: 2
Col 'category_refugees' unique values: 2
Col 'category_death' unique values: 2
Col 'category_other_aid' unique values: 2
Col 'category_infrastructure_related' unique values: 2
Col 'category_transport' unique values: 2
Col 'category_buildings' unique values: 2
Col 'category_electricity' unique values: 2
Col 'category_tools' un

Dropping `child_alone` column as it has always the same value.

In [20]:
df_data.drop(columns=["category_child_alone"], inplace=True)

#### Binarize values

In [21]:
for col in df_categories_new.columns:
    if col in df_data:
        df_data[col] = df_data[col].apply(lambda x: 1 if x >= 1 else 0)
        print("Col '{}' unique values: {}".format(col, df_data[col].nunique()))

Col 'category_related' unique values: 2
Col 'category_request' unique values: 2
Col 'category_offer' unique values: 2
Col 'category_aid_related' unique values: 2
Col 'category_medical_help' unique values: 2
Col 'category_medical_products' unique values: 2
Col 'category_search_and_rescue' unique values: 2
Col 'category_security' unique values: 2
Col 'category_military' unique values: 2
Col 'category_water' unique values: 2
Col 'category_food' unique values: 2
Col 'category_shelter' unique values: 2
Col 'category_clothing' unique values: 2
Col 'category_money' unique values: 2
Col 'category_missing_people' unique values: 2
Col 'category_refugees' unique values: 2
Col 'category_death' unique values: 2
Col 'category_other_aid' unique values: 2
Col 'category_infrastructure_related' unique values: 2
Col 'category_transport' unique values: 2
Col 'category_buildings' unique values: 2
Col 'category_electricity' unique values: 2
Col 'category_tools' unique values: 2
Col 'category_hospitals' uniq

### Saving the clean dataset into an sqlite database

In [22]:
!rm Disasters.db

rm: Disasters.db: No such file or directory


In [23]:
engine = create_engine("sqlite:///{}".format(DATA_OUTPUT_FILENAME))
df_data.to_sql("{}".format(SQL_TABLE_NAME), engine, index=False)

In [24]:
!ls

Disasters.db       ETL Pipeline.ipynb ML Pipeline.ipynb  model.pkl
